In [ ]:
from math import sqrt
from numpy import array
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
#aws s3 sync s3://remote_S3_bucket local_directory
#training file data plus additional features (category ID and city code) saved in previous, exploratory section
dataset_2D_all = read_csv('sales_categorized.csv',sep=',',skiprows=1,names=['date','date_block_num','shop_id','item_id','item_price','item_cnt_day','city_encoded','item_category_id'])
dataset_2D_all.head()

In [ ]:
dataset_2D_all.isnull().sum()
#for more advanced approach, null imputing should be provided or one should choose the model which can hadle missing values
#for the first attempt I'm choosing removing these noisy variables
#dataset for univariate attempt
#split prepare dataset for univariate analysis
dataset_2D_item_cnt = dataset_2D_all.iloc[: ,5]
dataset_2D_grouped = dataset_2D_all.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day':'sum'}).rename(columns={'item_cnt_day': 'cnt_sum_M'})
dataset_2D_grouped.dtypes

In [ ]:
cnt_sum_M_grouped = dataset_2D_grouped['cnt_sum_M']
cnt_sum_M_grouped.head()

In [ ]:
type(cnt_sum_M_grouped)
#pandas.core.series.Series
cnt_sum_M_grouped.shape
#(1609124,)

In [2]:
#Given the original shape of the data, they couldn't be easily convert into 3D array - when you divide 1609124 by nr of suggested for LSTM nr of samples you can't get an integer.
#Maybe there is another, more direct solution
#I decided to find the closest lenght of the dataset which can be divided by any divisor in range(200,300)
#It enables to keep 99.99% of the data

In [ ]:
#My n value calculator used to calculate integer closest to the original n value
n=1609124
z=list(range(200,300))
for i in range(n-1000,n-1):
    for l in range(200,300):
        if(i%l==0):
            print(i)
            print(l)
#1609120
#226

In [ ]:
#Let's reshape it to 3D dataset
import numpy as np
data_2D = np.array(cnt_sum_M_grouped.iloc[4:])
n = 1609120
length = 226
samples = list()

for i in range(0,n,length):
    sample = data_2D[i:i+length]
    samples.append(sample)
    
# reshape into [samples, timesteps, features]
rounded = int(round(len(samples)))
data_3D = data_2D.reshape((rounded, length, 1))
print(data_3D.shape)
#(7120, 226, 1)

In [ ]:
import pandas as pd
data_2D_series = pd.Series(data_2D)
data_int_series=data_2D_series.astype(int)

In [ ]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# transform list into supervised learning format
def series_to_supervised(data, n_in, n_out=1):
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	agg.dropna(inplace=True)
	return agg.values

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# difference dataset
def difference(data, order):
	return [data[i]-data[i-order] for i in range(order, len(data))]

# fit a model
def model_fit(train, config):
	# unpack config
	n_input, n_nodes, n_epochs, n_batch, n_diff = config
	# prepare data
	if n_diff > 0:
		train = difference(train, n_diff)
	# transform series into supervised format
	data = series_to_supervised(train, n_in=n_input)
	# separate inputs and outputs
	train_x, train_y = data[:, :-1], data[:, -1]
	# reshape input data into [samples, timesteps, features]
	n_features = 1
	train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
	# define model
	model = Sequential()
	model.add(LSTM(n_nodes, activation='relu', input_shape=(n_input, n_features)))
	model.add(Dense(n_nodes, activation='relu'))
	model.add(Dense(1))
	model.compile(loss='mse', optimizer='adam')
	# fit model
	model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
	return model

# forecast with the fit model
def model_predict(model, history, config):
	# unpack config
	n_input, _, _, _, n_diff = config
	# prepare data
	correction = 0.0
	if n_diff > 0:
		correction = history[-n_diff]
		history = difference(history, n_diff)
	# reshape sample into [samples, timesteps, features]
	x_input = array(history[-n_input:]).reshape((1, n_input, 1))
	# forecast
	yhat = model.predict(x_input, verbose=0)
	return correction + yhat[0]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# fit model
	model = model_fit(train, cfg)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = model_predict(model, history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	print(' > %.3f' % error)
	return error

# score a model, return None on failure
def repeat_evaluate(data, config, n_test, n_repeats=10):
	# convert config to a key
	key = str(config)
	# fit and evaluate the model n times
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	# summarize score
	result = mean(scores)
	print('> Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test):
	# evaluate configs
	scores = scores = [repeat_evaluate(data, cfg, n_test) for cfg in cfg_list]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a list of configs to try
def model_configs():
	# define scope of configs
	n_input = [12]
	n_nodes = [100]
	n_epochs = [50]
	n_batch = [1, 150]
	n_diff = [12]
	# create configs
	configs = list()
	for i in n_input:
		for j in n_nodes:
			for k in n_epochs:
				for l in n_batch:
					for m in n_diff:
						cfg = [i, j, k, l, m]
						configs.append(cfg)
	print('Total configs: %d' % len(configs))
	return configs


In [ ]:
# define dataset
series = data_int_series
data = series.values
# data split
n_test = 12
# model configs
cfg_list = model_configs()
# grid search
scores = grid_search(data, cfg_list, n_test)
print('done')
# list top 10 configs
for cfg, error in scores[:3]:
	print(cfg, error)